In [90]:
##########################    POPULATION RELATED
import pandas as pd
import numpy as np
min_agegroup=5
max_agegroup=14
male_groups=[1,3,5,7]
black_groups=[3,4]
hisp_groups=[2]

for column_name_no in range(1990, 1991):
    file = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_data.csv', header=0)
    n=str(column_name_no%100)
    if column_name_no%100<10:
        n="0"+str(column_name_no%100)
    unique_counties2=file["FIPS"+n].unique()
    unique_counties=np.delete(unique_counties2, [0])
    data1 = np.zeros((len(unique_counties),7),dtype=float)
    filename=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\economic_bureau\stch"+str(column_name_no-1)+".csv"
    filename2=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\economic_bureau\stch"+str(column_name_no-2)+".csv"
    year_file = pd.read_csv(filename)
    year_file2 = pd.read_csv(filename2)
    if column_name_no==1990:
        year_file3 = pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\economic_bureau\stch1990.csv")
        for cnt in range(0,len(unique_counties)):
            data1[cnt,0]=int(unique_counties[cnt])
            file_county=year_file[(year_file["FIPS"]==unique_counties[cnt])]
            file_county2=year_file2[(year_file2["FIPS"]==unique_counties[cnt])]
            pop_pre=0
            for cnt2 in range(1,19):
                data1[cnt,1]=data1[cnt,1] + int(file_county["P"+str(cnt2)].sum())
                pop_pre=pop_pre + int(file_county2["P"+str(cnt2)].sum())
                if cnt2>4 and cnt2<15:
                    data1[cnt,3]=data1[cnt,3] + int(file_county["P"+str(cnt2)].sum())
                data1[cnt,4]=data1[cnt,4]+int(file_county.iloc[0]["P"+str(cnt2)])+int(file_county.iloc[2]["P"+str(cnt2)])+int(file_county.iloc[4]["P"+str(cnt2)])
                data1[cnt,5]=data1[cnt,5]+int(file_county.iloc[2]["P"+str(cnt2)])+int(file_county.iloc[3]["P"+str(cnt2)])
            file_county3=year_file3[(year_file3["county"]==unique_counties[cnt])]
            pop90=int(file_county3.sum()[3])
            file_hisp=file_county3[file_county3["ethnic"].isin(hisp_groups)]
            data1[cnt,6]=int(file_hisp.sum()[3])*100/pop90
            if pop_pre==0:
                data1[cnt,2]=0
            else:
                data1[cnt,2]=(data1[cnt,1]-pop_pre)*100/pop_pre
            if data1[cnt,1]==0:
                for cnt2 in range(3,7):
                    data1[cnt,cnt2]==0            
            else:
                data1[cnt,3]=data1[cnt,3]*100/data1[cnt,1]
                data1[cnt,4]=data1[cnt,4]*100/data1[cnt,1]
                data1[cnt,5]=data1[cnt,5]*100/data1[cnt,1]
    else:
        for cnt in range(0,len(unique_counties)):
            data1[cnt,0]=int(unique_counties[cnt])
            file_county=year_file[(year_file["county"]==unique_counties[cnt])]
            data1[cnt,1]=int(file_county.sum()[3])            
            if column_name_no==1991:
                file_county2=year_file2[(year_file2["FIPS"]==unique_counties[cnt])]
                pop_pre=0
                for cnt2 in range(1,19):
                    pop_pre=pop_pre + int(file_county2["P"+str(cnt2)].sum())
            else:
                file_county2=year_file2[(year_file2["county"]==unique_counties[cnt])]
                pop_pre=int(file_county2.sum()[3]) 
            if pop_pre==0:
                data1[cnt,2]=0
            else:
                data1[cnt,2]=(data1[cnt,1]-pop_pre)*100/pop_pre
            file_age=year_file[(year_file["county"]==unique_counties[cnt]) & (year_file["agegroup"]>=min_agegroup) &(year_file["agegroup"]<=max_agegroup)]
            file_male=year_file[(year_file["county"]==unique_counties[cnt]) & (year_file["racesex"].isin(male_groups))]
            file_black=year_file[(year_file["county"]==unique_counties[cnt]) & (year_file["racesex"].isin(black_groups))]
            file_hisp=year_file[(year_file["county"]==unique_counties[cnt]) & (year_file["ethnic"].isin(hisp_groups))]
            if data1[cnt,1]==0:
                for cnt2 in range(3,7):
                    data1[cnt,cnt2]==0            
            else:
                data1[cnt,3]=int(file_age.sum()[3])*100/data1[cnt,1]
                data1[cnt,4]=int(file_male.sum()[3])*100/data1[cnt,1]
                data1[cnt,5]=int(file_black.sum()[3])*100/data1[cnt,1]
                data1[cnt,6]=int(file_hisp.sum()[3])*100/data1[cnt,1]
    go=pd.DataFrame(data=data1,columns=['COUNTIES','POP','POP_INC','POP_WORK_AGE','POP_MALE','POP_BLACK','POP_HISP'])
    filename=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Yearly data 5 csv\\5economic_bureau_"+str(column_name_no)+".csv"
    go.to_csv(filename)

In [79]:
###################################   OTHER FEATURES
import pandas as pd
import numpy as np
import addfips
af=addfips.AddFIPS()
ngroups=5
inc_exceptions=[1995,1997]
inc_exceptions2=[1991,1992,1993]
file_coll=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\education.csv", usecols=["FIPS","1990","2000"])
file_hs=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\high_school.csv", usecols=["FIPS","1990","2000"])
file_air=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\airport.csv",parse_dates=['ActivationDate'])
fips_codes=np.zeros((int(file_air.shape[0]),1))
inflation=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inflationrate.csv")
for cnt in range(0,int(file_air.shape[0])):
    fips_codes[cnt]=(af.get_county_fips(file_air.iloc[cnt][1], file_air.iloc[cnt][0]))
fip=pd.DataFrame(data=fips_codes)
file_air["FIPS"]=fip
file_air["YEAR"]=file_air['ActivationDate'].dt.year    
file = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_data.csv', header=0)
for column_name_no in range(1990, 1994):
    inflation1=inflation[inflation["YEARS"]==column_name_no-1]
    inf_rate = inflation1["VALUE"].values[0]
    n=str((column_name_no)%100)
    if (column_name_no)%100<10:
        n="0"+str((column_name_no)%100)
    unique_counties2=file["FIPS"+n].unique()
    unique_counties=np.delete(unique_counties2, [0])    
    data1 = np.zeros((len(unique_counties),9))    
    coll_col="2000"
    if column_name_no-1<2000:
        coll_col="1990"
    ref = (column_name_no-1)%100
    unempname = r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\laucnty" + str(ref) + ".csv"
    filename_area=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\area"+str(int(np.floor(((column_name_no-1)/100)%10)))+"0.csv"
    filename_rent=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\rentownership"+str(int(np.floor((column_name_no-1)/10)))+"0.csv"
    if ref<10:
        unempname = r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\laucnty0" + str(ref) + ".csv"  
    if ref==89:
        unempname = r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\laucnty90.csv"
        filename_area=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\area90.csv"
        filename_rent=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\rentownership1990.csv"
    file_area=pd.read_csv(filename_area, encoding='latin-1')
    fips_codes2=np.zeros((int(file_area.shape[0]),1))
    for cnt in range(0,int(file_area.shape[0])):
        if file_area.iloc[cnt][5]=="St. Louis city":
            fips_codes2[cnt]=29510
        if ("County" in file_area.iloc[cnt][5])==False:
            state=file_area.iloc[cnt][5]
        else:   
        #if ("County" in file_area.iloc[cnt][5])==True:
            fips_codes2[cnt]=(af.get_county_fips(file_area.iloc[cnt][5], state))
    fip2=pd.DataFrame(data=fips_codes2)
    file_area["FIPS"]=fip2    
    filename_minc=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inc_"+str(column_name_no-1)+".csv"
    if column_name_no in inc_exceptions:
        filename_minc=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inc_"+str(column_name_no-2)+".csv"
    if column_name_no in inc_exceptions2:
        filename_minc=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inc_1989.csv"
    file_minc=pd.read_csv(filename_minc, usecols=["state_code","county_code","med_inc"])    
    file_rent=pd.read_csv(filename_rent)
    fips_codes3=np.zeros((int(file_rent.shape[0]),1))
    for cnt in range(0,int(file_rent.shape[0])):
        fips_codes3[cnt]=int(af.get_county_fips(file_rent.iloc[cnt]["County"], file_rent.iloc[cnt]["State"]))
    fip3=pd.DataFrame(data=fips_codes3)
    file_rent["FIPS"]=fip3    
    file_homeownership = file_rent    
    if (column_name_no-1)>=2010:
        file_rent=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\rent2010.csv", encoding='latin-1')    
    for cnt in range(0,len(unique_counties)):
        if int(unique_counties[cnt]/1000) not in [20,29,40,8]:
            continue
        data1[cnt,0]=(unique_counties[cnt])        
        file_coll["FIPS2"]=(file_coll["FIPS"].values).tolist()
        file_coll_county=file_coll[(file_coll["FIPS2"]==unique_counties[cnt])]
        data1[cnt,1]=(file_coll_county.iloc[0][coll_col])        
        file_hs["FIPS2"]=(file_hs["FIPS"].values).tolist()
        file_hs_county=file_hs[(file_hs["FIPS2"]==unique_counties[cnt])]
        data1[cnt,2]=(file_hs_county.iloc[0][coll_col])        
        file_ur=pd.read_csv(unempname, usecols=["FIPS","Unemployment Rate"])
        file_ur_county=file_ur[((file_ur["FIPS"])==int(unique_counties[cnt]))]
        data1[cnt,3]=(file_ur_county.iloc[0]["Unemployment Rate"])    
        file_air_county=file_air[(file_air["FIPS"]==unique_counties[cnt]) & (file_air["YEAR"]<column_name_no)]
        data1[cnt,4]=int(file_air_county.shape[0])        
        file_area_county=file_area[(file_area["FIPS"]==unique_counties[cnt])]
        data1[cnt,5]=(file_area_county.iloc[0]["Sq.Mi."])        
        file_minc_county=file_minc[((file_minc["state_code"])==int(unique_counties[cnt]/1000)) & ((file_minc["county_code"])==int(unique_counties[cnt]%1000))]
        data1[cnt,6]=float(file_minc_county.iloc[0]["med_inc"])*inf_rate        
        file_rent_county=file_rent[(file_rent["FIPS"]==unique_counties[cnt])]
        data1[cnt,7]=float(file_rent_county.iloc[0]["Median Contract Rent"])*inf_rate
        file_homeownership_county=file_homeownership[(file_homeownership["FIPS"]==unique_counties[cnt])]
        data1[cnt,8]=(file_homeownership_county.iloc[0]["Ownership"])        
    go=pd.DataFrame(data=data1,columns=['COUNTIES','COLLEGE_COMP','HIGHSCHOOL_COMP','UNEMPLOYMENT_RATE','NAIRPORTS','LAND_AREA','MED_INC','MED_RENT','HOME_OWNERSHIP'])
    filename=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Yearly data 5 csv\\5features1_"+str(column_name_no-1)+".csv"
    go.to_csv(filename)

In [78]:
##############################    WAGE
import pandas as pd
import numpy as np
import us
ngroups=5
group2=[["Construction","Manufacturing"],"Trade, transportation, and utilities",["Other services","Education and health services","Professional and business services","Financial activities","Information"],"Leisure and hospitality","Trade, transportation, and utilities"]
file = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_data.csv', header=0)
inflation=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inflationrate.csv")
for column_name_no in range(1990, 1994):
    inflation1=inflation[inflation["YEARS"]==column_name_no-1]
    inf_rate = inflation1["VALUE"].values[0]
    n=str(column_name_no%100)
    if column_name_no%100<10:
        n="0"+str(column_name_no%100)
    unique_counties2=file["FIPS"+n].unique()
    unique_counties=unique_counties2[unique_counties2!=0]
    data1 = np.zeros((len(unique_counties)*ngroups,3))    
    filename_wage=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\wage"+str((column_name_no-1)%100)+".csv"
    if ((column_name_no-1)%100)<10:
        filename_wage=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\wage0"+str((column_name_no-1)%100)+".csv"
    if column_name_no==1990:
        filename_wage=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\wage"+str((column_name_no)%100)+".csv"
    file_wage=pd.read_csv(filename_wage, usecols=["Area Code","Industry","Annual Average Pay", "St Name"])
    for cnt in range(0,len(unique_counties)):
        county_name=(int(unique_counties[cnt]))
        file_ur_county=file_wage[((file_wage["Area Code"])==county_name)]
        if file_ur_county.shape[0]==0:
            t = str(int(county_name/1000))
            if (int(county_name/1000))<10:
                t = "0" + str(int(county_name/1000))
            state = str(us.states.lookup(t))
            file_ur_county=file_wage[((file_wage["St Name"])==state)]
            
        for cnt3 in range(1,ngroups+1):
            data1[(cnt*ngroups)+cnt3-1,0]=unique_counties[cnt]
            data1[(cnt*ngroups)+cnt3-1,1]=cnt3
            if type(group2[cnt3-1]) == list:
                wage=0
                for cnt2 in range(0,len(group2[cnt3-1])):
                    file_ur_county2=file_ur_county[(file_ur_county["Industry"])==group2[cnt3-1][cnt2]]
                    if file_ur_county2.shape[0]!=0:
                        wage=wage+int(file_ur_county2.iloc[0]["Annual Average Pay"].replace(',',''))    
                    else:
                        wage=wage+int(file_ur_county.iloc[0]["Annual Average Pay"].replace(',',''))
                data1[(cnt*ngroups)+cnt3-1,2]=(wage*inf_rate)/len(group2[cnt3-1])
            else:
                file_ur_county2=file_ur_county[(file_ur_county["Industry"])==group2[cnt3-1]]
                if file_ur_county2.shape[0]!=0:
                    data1[(cnt*ngroups)+cnt3-1,2]=int(file_ur_county2.iloc[0]["Annual Average Pay"].replace(',',''))*inf_rate    
                else:
                    data1[(cnt*ngroups)+cnt3-1,2]=int(file_ur_county.iloc[0]["Annual Average Pay"].replace(',',''))*inf_rate
                        
    go=pd.DataFrame(data=data1,columns=['COUNTIES','GROUPS','AVG_WAGE'])
    filename=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Yearly data 5 csv\\5feature_wage"+str(column_name_no)+".csv"
    go.to_csv(filename)

In [23]:
##############################    WAGE : 20 GROUPS
import pandas as pd
import numpy as np
import us
ngroups=20
group2=["Natural resource and mining","Natural resource and mining","Trade, transportation, and utilities","Construction","Manufacturing","Trade, transportation, and utilities","Trade, transportation, and utilities","Trade, transportation, and utilities","Information","Financial activities","Service-providing","Professional and business services","Professional and business services","Professional and business services","Education and health services","Education and health services","Leisure and hospitality","Leisure and hospitality","Other services","Other services"]
file = pd.read_csv(r'\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Organised Derived CSVs\400_data.csv', header=0)
inflation=pd.read_csv(r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\inflationrate.csv")
for column_name_no in range(1990, 2013):
    inflation1=inflation[inflation["YEARS"]==column_name_no-1]
    inf_rate = inflation1["VALUE"].values[0]
    n=str(column_name_no%100)
    if column_name_no%100<10:
        n="0"+str(column_name_no%100)
    unique_counties2=file["FIPS"+n].unique()
    unique_counties=unique_counties2[unique_counties2!=0]
    data1 = np.zeros((len(unique_counties)*ngroups,3))    
    filename_wage=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\wage"+str((column_name_no-1)%100)+".csv"
    if ((column_name_no-1)%100)<10:
        filename_wage=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\wage0"+str((column_name_no-1)%100)+".csv"
    if column_name_no==1990:
        filename_wage=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Preprocessing_files\\wage"+str((column_name_no)%100)+".csv"
    file_wage=pd.read_csv(filename_wage, usecols=["Area Code","Industry","Annual Average Pay", "St Name"])
    for cnt in range(0,len(unique_counties)):
        county_name=(int(unique_counties[cnt]))
        file_ur_county=file_wage[((file_wage["Area Code"])==county_name)]
        if file_ur_county.shape[0]==0:
            t = str(int(county_name/1000))
            if (int(county_name/1000))<10:
                t = "0" + str(int(county_name/1000))
            state = str(us.states.lookup(t))
            file_ur_county=file_wage[((file_wage["St Name"])==state)]
        for cnt3 in range(1,ngroups+1):
            data1[(cnt*ngroups)+cnt3-1,0]=unique_counties[cnt]
            data1[(cnt*ngroups)+cnt3-1,1]=cnt3
            file_ur_county2=file_ur_county[(file_ur_county["Industry"])==group2[cnt3-1]]
            if file_ur_county2.shape[0]!=0:
                data1[(cnt*ngroups)+cnt3-1,2]=int(file_ur_county2.iloc[0]["Annual Average Pay"].replace(',',''))*inf_rate    
            else:
                data1[(cnt*ngroups)+cnt3-1,2]=int(file_ur_county.iloc[0]["Annual Average Pay"].replace(',',''))*inf_rate
    go=pd.DataFrame(data=data1,columns=['COUNTIES','GROUPS','AVG_WAGE'])
    filename=r"\\nas01.itap.purdue.edu\puhome\My Documents\Research\Paper1\Yearly data 5 csv\\20feature_wage"+str(column_name_no)+".csv"
    go.to_csv(filename)

In [1]:
group2=["Natural resource and mining","Natural resource and mining","Trade, transportation, and utilities","Construction","Manufacturing","Trade, transportation, and utilities","Trade, transportation, and utilities","Trade, transportation, and utilities","Information","Financial activities","Service-providing","Professional and business services","Professional and business services","Professional and business services","Education and health services","Education and health services","Leisure and hospitality","Leisure and hospitality","Other services","Other services"]
len(group2)

20